In [1]:
!pip install -q langgraph langchain langchain-google-genai langchain-community faiss-cpu sentence-transformers requests

In [2]:
from typing import Annotated, TypedDict, Literal
import requests
import random

In [6]:
!pip install -q langchain langgraph langchain-groq

In [7]:
!pip install -q python-dotenv

In [12]:
from dotenv import load_dotenv
import os

load_dotenv()  # <-- THIS LINE WAS MISSING

True

In [14]:
from langchain_groq import ChatGroq

In [15]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",  # or mixtral-8x7b
    api_key=os.environ["GROQ_API_KEY"]
)

llm_with_tools = llm.bind_tools([get_weather])


In [16]:
from langchain_core.tools import tool

@tool
def get_weather(city: str):
    """Get the current weather for a city using Open-Meteo."""
    try:
        geo_url = "https://geocoding-api.open-meteo.com/v1/search"
        geo_data = requests.get(
            geo_url,
            params={"name": city, "count": 1, "format": "json"}
        ).json()

        if not geo_data.get("results"):
            return f"City {city} not found."

        lat = geo_data["results"][0]["latitude"]
        lon = geo_data["results"][0]["longitude"]

        weather_url = "https://api.open-meteo.com/v1/forecast"
        w_data = requests.get(
            weather_url,
            params={"latitude": lat, "longitude": lon, "current": "temperature_2m"}
        ).json()

        temp = w_data["current"]["temperature_2m"]
        return f"The weather in {city} is {temp}°C."
    except Exception as e:
        return f"Error: {e}"

In [17]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [18]:
def chatbot_node(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [19]:
tool_node = ToolNode(tools=[get_weather])

In [20]:
def should_continue(state: State) -> Literal["tools", END]:
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [21]:
builder = StateGraph(State)

builder.add_node("chatbot", chatbot_node)
builder.add_node("tools", tool_node)

builder.add_edge(START, "chatbot")
builder.add_conditional_edges(
    "chatbot",
    should_continue,
    {"tools": "tools", END: END}
)
builder.add_edge("tools", "chatbot")

graph = builder.compile()
print("✅ Weather Agent Ready")

✅ Weather Agent Ready


In [22]:
inputs = {"messages": [("user", "What is the weather in London?")]}

for event in graph.stream(inputs):
    for node, value in event.items():
        msg = value["messages"][-1]
        print(f"{node}: {msg.content}")

chatbot: 
tools: The weather in London is 0.6°C.
chatbot: However, the get_weather function was called, but the output of the function was not provided. To provide the output, the get_weather function would need to be implemented.

Here is an example implementation of the get_weather function using the Open-Meteo API:

```python
import requests

def get_weather(city):
    api_url = f"http://api.open-meteo.com/v1/forecast?current_weather=true&hourly=temperature_2m&daily=wr_time,wr_summary&current_weather=temperature_2m&hourly=temperature_2m&daily=wr_time,wr_summary&timezone=Europe%2FLondon&current_weather=true&hourly=temperature_2m&current_weather=true&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=temperature_2m&hourly=tem